In [66]:
import pymc as pm
from pymc.ode import DifferentialEquation
import arviz as az
import numpy as np

In [67]:
pm.__version__

'4.0.0'

* https://docs.pymc.io/en/v3/pymc-examples/examples/ode_models/ODE_with_manual_gradients.html
* https://mc-stan.org/users/documentation/case-studies/lotka-volterra-predator-prey.html#mechanistic-model-the-lotka-volterra-equations

# Data


In [68]:
year = np.arange(1900, 1921, 1)
year = np.arange(0, 21)

hare = np.array([30.0, 47.2, 70.2, 77.4, 36.3, 20.6, 18.1, 21.4, 22.0, 25.4,
                 27.1, 40.3, 57.0, 76.6, 52.3, 19.5, 11.2, 7.6, 14.6, 16.2, 24.7])
lynx = np.array([4.0, 6.1, 9.8, 35.2, 59.4, 41.7, 19.0, 13.0, 8.3, 9.1, 7.4,
                8.0, 12.3, 19.5, 45.7, 51.1, 29.7, 15.8, 9.7, 10.1, 8.6])

Y = np.vstack((Hare, Lynx)).T
Y

array([[30. ,  4. ],
       [47.2,  6.1],
       [70.2,  9.8],
       [77.4, 35.2],
       [36.3, 59.4],
       [20.6, 41.7],
       [18.1, 19. ],
       [21.4, 13. ],
       [22. ,  8.3],
       [25.4,  9.1],
       [27.1,  7.4],
       [40.3,  8. ],
       [57. , 12.3],
       [76.6, 19.5],
       [52.3, 45.7],
       [19.5, 51.1],
       [11.2, 29.7],
       [ 7.6, 15.8],
       [14.6,  9.7],
       [16.2, 10.1],
       [24.7,  8.6]])

In [69]:
times = year.shape[0]
times

21

# ODE
https://www.pymc.io/projects/docs/en/stable/api/generated/pymc.ode.DifferentialEquation.html  
https://docs.pymc.io/en/v3/pymc-examples/examples/ode_models/ODE_API_introduction.html

In [70]:
def Lotka_Volterra(y, t, p):
    # p = [alpha, beta, gamma, delta]
    # y = [u,v]
    
    prey = (p[0] - p[1]*y[1]) * y[0]
    predator = (-p[2] + p[3]*y[0]) * y[1]

    return [prey, predator]


Lotka_Volterra_ode = DifferentialEquation(func=Lotka_Volterra, times=year, n_states=2, n_theta=4, t0=0)

In [72]:
with pm.Model() as LV_model:

    # Priors for unknown model parameters

    alpha = pm.Normal("alpha", mu=1, sigma=0.5)
    beta = pm.Normal("beta", mu=0.05, sigma=0.05)
    
    gamma = pm.Normal("gamma", mu=1, sigma=0.5)
    delta = pm.Normal("delta", mu=0.05, sigma=0.05)

    
    prey_t0 = pm.Lognormal("prey_t0", mu=np.log(10), sigma=1)
    predator_t0 = pm.Lognormal("predator_t0", mu=np.log(10), sigma=1)
    sigma = pm.Lognormal("sigma", mu=-1, sigma=1, shape=2)    

    p = [alpha, beta, gamma, delta]
    
    mu = Lotka_Volterra_ode(y0=[prey_t0, predator_t0], theta = p)
    
    
    Y_obs = pm.Lognormal("Y_obs", mu=pm.math.log(mu), sigma=sigma, observed=Y)
        
    trace = pm.sample()

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
/home/canyon/miniconda3/envs/intuitive_bayes/lib/python3.10/site-packages/scipy/integrate/_odepack_py.py:247: ODEintWarning: Excess work done on this call (perhaps wrong Dfun type). Run with full_output = 1 to get quantitative information.
  warnings.warn(warning_msg, ODEintWarning)
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [alpha, beta, gamma, delta, prey_t0, predator_t0, sigma]


KeyError: 0